Building an Artificial Neural Network model (churn model)

Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
tf.__version__

'2.7.0'

Part-1 Data Processing

Importing data

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


Encoding Catogorial data

Label encoded the 'Gender' column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])


In [ ]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Here as you can see, we have used LabelEncoder
1 = Female
0 = Male

One hot encoding the geography column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


so here as you can see, we have used OneHotEncoder in which
1.0 0.0 0.0 = France
0.0 1.0 0.0 = Germany
0.0 0.0 1.0 = Spain 

Splitting the dataset into traing test and test test

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0) 

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

Part2 - Building the ANN 

Intialize The ANN

In [ ]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

Part-3 Training the ANN

Compiling The ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Training the ANN

In [ ]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6727 - accuracy: 0.6283
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4712 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4361 - accuracy: 0.7969
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4225 - accuracy: 0.8076
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4108 - accuracy: 0.8161
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4015 - accuracy: 0.8227
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3923 - accuracy: 0.8324
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3836 - accuracy: 0.8371
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3761 - accuracy: 0.8428
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3700 - accura

out of 100, you have 86 correct prediction 

Part 4 Making the prediction.

Predticting the result

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


predticting the test result

In [ ]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


using the confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred) 

[[1535   60]
 [ 212  193]]


0.864

So our model is 86% accurate, 

1535 = correct prediction that the customer stayed in the bank

212 = inncorrect prediction that the customer stayed in the bank

60 = incorrect predtiction that the customer leave the bank

193 = correct predtiction that the customer leaves the bank